In [ ]:
# !pip install duckdb pandarallel ipywidgets

In [ ]:
import duckdb

excape_path = './data/Full-ExCAPE.tsv'

con = duckdb.connect(database=':memory:', read_only=False)
con.execute(f'CREATE TABLE excape AS SELECT * FROM read_csv_auto(\'{excape_path}\')')

con.execute('SELECT * FROM excape LIMIT 10').fetchdf()

In [ ]:
print(f'Number of rows: {con.execute("SELECT COUNT(*) FROM excape").fetchdf().values[0][0]:,}')

unique_genes = con.execute('SELECT DISTINCT Gene_Symbol FROM excape').fetchdf()

print(f'Number of unique genes: {unique_genes.shape[0]:,}')
unique_genes.sample(5)

In [ ]:
# Save pXC50 activity dataset
from smiles_utils import clean_smile
import pandas as pd
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True, nb_workers=6)


idx = 0
min_assays_threshold = 500
pXC50_activity_dataset = pd.DataFrame(columns=["SMILES", "pXC50", "Gene", "Gene_idx"])

for gene in unique_genes["Gene_Symbol"]:
    gene_rows = con.execute(f'SELECT * FROM excape WHERE Gene_Symbol = \'{gene}\'').fetchdf()

    # Clean and remove duplicates
    gene_rows = gene_rows.dropna(subset=["SMILES"]).drop_duplicates(subset=["SMILES"])
    assay_pXC50 = gene_rows[["SMILES", "pXC50"]].copy()
    assay_pXC50["pXC50"] = assay_pXC50["pXC50"].astype(float)
    assay_pXC50 = assay_pXC50.dropna(subset=["pXC50"])

    # First skip: not enough assays to clean the SMILES
    if len(assay_pXC50) < min_assays_threshold:
        continue

    assay_pXC50["SMILES"] = assay_pXC50["SMILES"].parallel_apply(clean_smile)
    assay_pXC50 = assay_pXC50.dropna(subset=["SMILES"]).drop_duplicates(subset=["SMILES"])
    assay_pXC50["pXC50"] = assay_pXC50["pXC50"].clip(0, 10)
    
    # Second skip: cleaning the SMILES filtered out invalid strings
    if len(assay_pXC50) < min_assays_threshold:
        continue

    assay_pXC50["Gene"] = gene
    assay_pXC50["Gene_idx"] = idx
    pXC50_activity_dataset = pd.concat([pXC50_activity_dataset, assay_pXC50], ignore_index=True)
    idx += 1

pXC50_activity_dataset.to_csv(f"./data/pXC50_activity_dataset_over_{min_assays_threshold}.csv", index=False)
print(f"Number of genes with at least {min_assays_threshold} assays: {idx:,}")
print(f"Number of assays: {len(pXC50_activity_dataset):,}")
print(f"Number of unique SMILES: {pXC50_activity_dataset['SMILES'].nunique():,}")
print(f"Number of unique genes: {pXC50_activity_dataset['Gene'].nunique():,}")
pXC50_activity_dataset.sample(5)

In [30]:
# Create train/val/test split column
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split

np.random.seed(42)
random.seed(42)

SPLIT = True
min_assays_threshold = 500
all_assays = pd.read_csv(f"./data/pXC50_activity_dataset_over_{min_assays_threshold}.csv")

pivoted_assays = all_assays.pivot_table(index="SMILES", values="pXC50", columns="Gene")
pivoted_assays

Gene,ABCB1,ABL1,ACE,ACHE,ADAM10,ADAM17,ADORA1,ADORA2A,ADORA2B,ADORA3,ADRA1A,ADRA1B,ADRA1D,ADRA2A,ADRA2C,ADRB1,ADRB2,ADRB3,AGTR1,AGTR2,AHR,AKR1B1,AKT1,AKT2,AKT3,ALDH1A1,ALK,ALOX12,ALOX15,ALOX15B,ALOX5,ALPI,ALPL,ALPPL2,APAF1,APEX1,APLNR,APOBEC3A,APOBEC3F,APOBEC3G,AR,ATAD5,ATM,ATXN2,AURKA,AURKB,AVPR1A,AVPR1B,AVPR2,AXL,...,SLK,SMAD3,SMN1,SMN2,SMPD1,SNCA,SOAT1,SRC,SRMS,SRPK1,STAT3,STK17A,STK3,SYK,TAAR1,TACR1,TACR2,TACR3,TAOK1,TARDBP,TBK1,TBXA2R,TBXAS1,TDP1,TEK,TERT,TGFBR1,THRB,TNF,TNK2,TP53,TRPC4,TRPV1,TSG101,TSHR,TSPO,TSSK1B,TSSK2,TXNRD1,TYK2,TYMS,TYRO3,UBE2N,USP1,USP2,VDR,WRN,XBP1,YES1,ZAK
SMILES,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
BrC(=NNc1nn[nH]n1)c1ccncc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BrC(C=Cc1ccccc1)=NNc1nc(N2CCOCC2)nc(N2CCOCC2)n1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.13621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BrC(C=NN1CCN(Cc2ccccc2)CC1)=Cc1ccccc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.35,5.00,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.45,NaN,NaN,NaN,5.73622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BrC(C=NN1CCN(c2ccccc2)CC1)=Cc1ccccc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.93622,NaN,NaN,NaN,NaN,NaN,NaN,4.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.10,NaN,NaN,NaN,NaN
BrC(C=NN1CCN(c2ccccn2)CC1)=Cc1ccccc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.00,NaN,NaN,NaN,6.48625,NaN,NaN,NaN,NaN,NaN,NaN,5.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.05,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
c1nnc2c[nH]c(-c3cnc(N4CCCC4)c(N4CCCNCC4)n3)cc1-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c1nnn(C23CC4CC(CC(C4)C2)C3)n1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [37]:
def columns_with_less_than_N_assays(df, N):
    selected_columns =  df.columns[df.notnull().sum() < N]
    return len(selected_columns)

MIN_SPLIT_ASSAYS = 80
# Create column "split" indicating train/val/test
if SPLIT:
    train, test = train_test_split(pivoted_assays, test_size=0.2, random_state=42)
    train, val = train_test_split(train, test_size=0.2, random_state=42)
    train["split"] = "train"
    val["split"] = "val"
    test["split"] = "test"

    print("Columns with all nulls in each split:")
    print(pd.Series({
        "train": columns_with_less_than_N_assays(train, MIN_SPLIT_ASSAYS),
        "val": columns_with_less_than_N_assays(val, MIN_SPLIT_ASSAYS),
        "test": columns_with_less_than_N_assays(test, MIN_SPLIT_ASSAYS)
    }))
    

    pivoted_assays_split = pd.concat([train, val, test])

    print("\nProportion of train/val/test:")
    print(pd.DataFrame({
        "#": pivoted_assays_split["split"].value_counts().apply(lambda x: f"{x:,}"),
        "%": pivoted_assays_split["split"].value_counts(normalize=True)
    }))

    # pivoted_assays_split.to_csv(f"./data/pivoted_pXC50_over_{min_assays_threshold}_split.csv")

Columns with all nulls in each split:
train    0
val      9
test     0
dtype: int64

Proportion of train/val/test:
             #         %
split                   
train  420,460  0.639998
test   131,395  0.200001
val    105,116  0.160001
